In [3]:
import pandas as pd

In [14]:
# pull in only relevant data for development.
raw = pd.read_excel("../data/raw/MASTER FILE.xlsx", 
sheet_name="PAT_POP", 
index_col=0,
usecols=[3, 4, 5],
header=0)

In [ ]:
raw


,Unnamed: 1,SexName,RaceName,EthnicGroupName,HispanicOrNot,Age,FirstDXDate,IncludedEye,OCT_DATE,Sub-RPE 5mm OD,...,LDL date 30 days after FirstDxDate,LDL result 30 days after FirstDxDate,Potassium date 30 days after FirstDxDate,Potassium result 30 days after FirstDxDate,Serum Creatinine date 30 days after FirstDxDate,Serum Creatinine result 30 days after FirstDxDate,Sodium date 30 days after FirstDxDate,Sodium result 30 days after FirstDxDate,Total cholesterol date 30 days after FirstDxDate,Total cholesterol result 30 days after FirstDxDate
MRN,,,,,,,,,,,,,,,,,,,,,
2827913,NaN,Female,White,Caucasian,Not Hispanic,93.399041,2017-06-01,OU,NaN,0,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
2991136,NaN,Male,White,Caucasian,Not Hispanic,93.399041,2019-05-06,OU,NaN,0.7,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
2999099,NaN,Female,White,Caucasian,Not Hispanic,98.138261,2018-09-18,NaN,NaN,ONLY ONE IMAGE,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
4335406,NaN,Male,White,Caucasian,Not Hispanic,83.340177,2017-10-02,NaN,NaN,NO ATROPHY,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
4616162,NaN,Female,White,Caucasian,Not Hispanic,92.262833,2021-09-13,NaN,NaN,LESS THAN ONE YEAR FOLLOW UP,...,NaT,NaN,2021-10-08 15:49:00,4.4,2021-10-08 15:49:00,1.04,2021-10-08 15:49:00,138.0,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88420777,NaN,Female,White,Caucasian,Not Hispanic,91.066392,2021-11-01,NaN,NaN,BRVO,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
88494447,NaN,Female,White,Caucasian,Not Hispanic,79.983572,2022-01-14,NaN,NaN,LESS THAN ONE YEAR FOLLOW UP,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
89186919,NaN,Female,Unavailable,Other,Unavailable,77.234770,2022-01-26,NaN,NaN,LESS THAN ONE YEAR FOLLOW UP,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN
